In [1]:
import pandas as pd
import geopandas as gpd
import requests

In [2]:
def get_data(data_url: str, cbs = False):
    """Function to retrieve datasets from the internet

    Parameters
    ----------
    data_url : string
        The url that points to the dataset

    cbs: bool
        Whether the data is hosted by CBS

    Returns
    -------
    pd.DataFrame
        Dataframe containing data retrieved from data_url
    """    
    data = pd.DataFrame()
    if cbs:
        while data_url:
            r = requests.get(data_url).json()
            data = data.append(pd.DataFrame(r['value']))
            
            if '@odata.nextLink' in r:
                data_url = r['@odata.nextLink']
            else:
                data_url = None
    else:
        r = requests.get(data_url).json()
        data = data.append(pd.DataFrame(r))
    return data

In [3]:
covid19_link = 'https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.json'
vaccinaties_link = 'https://data.rivm.nl/data/covid-19/COVID-19_vaccinatiegraad_per_gemeente_per_week_leeftijd.json'
cbs_link = "https://beta-odata4.cbs.nl/CBS/70072ned"
filter_link = "$filter=startswith(RegioS, 'GM') and startswith(Perioden, '202')"

In [4]:
vaccinatie_df = get_data(vaccinaties_link)
vaccinatie_df.to_csv('Data/Vaccinaties_cumulatief.csv')

In [6]:
covid19_df = get_data(covid19_link)
covid19_df.to_csv('Data/COVID_19_cumulatief.csv')


In [69]:
# Filter to only download municipality data
cbs_kerncijfers = get_data(f"{cbs_link}/Observations?{filter_link}", cbs=True)

# Collect CBS metadata
groups = get_data(cbs_link + "/MeasureGroups", cbs = True)
codes = get_data(cbs_link + "/MeasureCodes", cbs=True)

In [68]:
# Add metadata for codes
cbs_kerncijfers_meta = pd.merge(cbs_kerncijfers, codes, left_on="Measure",right_on="Identifier")

,Id,Measure,ValueAttribute,Value,RegioS,Perioden
0,375789,M000352_3,Impossible,NaN,GM1680,1995JJ00
1,375790,3000,Impossible,NaN,GM1680,1995JJ00
2,375791,4000,Impossible,NaN,GM1680,1995JJ00
3,375792,40000_1,Impossible,NaN,GM1680,1995JJ00
4,375793,70200_1,Impossible,NaN,GM1680,1995JJ00
...,...,...,...,...,...,...
6998,5282787,A045820_3,Impossible,NaN,GM0193,2021JJ00
6999,5282788,A045823_3,Impossible,NaN,GM0193,2021JJ00
7000,5282789,GM000A,Impossible,NaN,GM0193,2021JJ00
7001,5282790,M000161,None,16.0,GM0193,2021JJ00


In [99]:
# Pivot van wide naar long 
cbs_kerncijfers_meta_pivot = cbs_kerncijfers_meta.pivot(index=['RegioS','Perioden'],columns=['Identifier','Title','Description'],values='Value')

In [100]:
cbs_kerncijfers_meta_pivot.to_csv('Data/CBS_kerncijfers_pivot.csv')

# Read and combine data

In [8]:
covid19_df = pd.read_csv('Data/COVID_19_cumulatief.csv')
vaccinatie_df = pd.read_csv('Data/Vaccinaties_cumulatief.csv')

cbs_kerncijfers_meta_pivot = pd.read_csv('Data/CBS_kerncijfers_pivot.csv',header=[1])
cbs_kerncijfers_meta_pivot = cbs_kerncijfers_meta_pivot.drop([0,1])

In [6]:
cbs_kerncijfers_klein = cbs_kerncijfers_meta_pivot[['Title','Totale bevolking','Mannen','Vrouwen','Jonger dan 5 jaar.1',
       '5 tot 10 jaar.1', '10 tot 15 jaar.1', '15 tot 20 jaar.1',
       '20 tot 25 jaar.1', '25 tot 45 jaar.1', '45 tot 65 jaar.1',
       '65 tot 80 jaar.1', '80 jaar of ouder.1','Nederlandse achtergrond.1',
       'Totaal met migratieachtergrond.1', 'Westerse migratieachtergrond.1',
       'Totaal niet-westerse migratieachtergrond.1', 'Marokko.1',
       '(voormalige) Nederlandse Antillen, Aruba.1', 'Suriname.1', 'Turkije.1',
       'Overig niet-westerse migratieachtergrond.1','Sterfte', 'Sterfte, relatief', 'Bevolkingsdichtheid',
       'Woningdichtheid', 'Koopwoningen', 'Huurwoningen']]

In [10]:
vaccinatie_df.head()

,Version,Date_of_report,Date_of_statistics,Region_level,Region_code,Region_name,Birth_year,Vaccination_coverage_partly,Vaccination_coverage_completed,Age_group
0,2,2021-11-10 10:30:00,2021-11-01,Gemeente,GM0014,Groningen,<=2009,83,81,12+
1,2,2021-11-10 10:30:00,2021-11-01,Gemeente,GM0014,Groningen,<2004,84,82,18+
2,2,2021-11-10 10:30:00,2021-11-01,Gemeente,GM0014,Groningen,2004-2009,68,65,12-17
3,2,2021-11-10 10:30:00,2021-11-01,Gemeente,GM0034,Almere,<=2009,74,72,12+
4,2,2021-11-10 10:30:00,2021-11-01,Gemeente,GM0034,Almere,<2004,76,75,18+


In [9]:
covid19_df_totalen = covid19_df[covid19_df['Date_of_report']=='2021-11-10 10:00:00']

In [11]:
vaccinatie_df_totalen = vaccinatie_df[vaccinatie_df['Date_of_statistics']=='2021-11-01']

In [13]:
vaccinatie_df_totalen = vaccinatie_df_totalen[vaccinatie_df_totalen['Age_group']=='18+']

In [15]:
vaccinatie_df_totalen = vaccinatie_df_totalen.loc[:,['Region_code','Vaccination_coverage_partly','Vaccination_coverage_completed']]

In [17]:
vaccinatie_df_totalen

,Region_code,Vaccination_coverage_partly,Vaccination_coverage_completed
1,GM0014,84,82
4,GM0034,76,75
7,GM0037,84,82
10,GM0047,83,81
13,GM0050,82,80
...,...,...,...
1117,VR21,88,87
1120,VR22,86,85
1123,VR23,85,84
1126,VR24,81,80


In [18]:
result_df_klein = covid19_df_totalen.merge(cbs_kerncijfers_klein,how='left',left_on='Municipality_code',right_on=('Title'))
result_df_klein = result_df_klein.merge(vaccinatie_df_totalen,how='left',left_on='Municipality_code',right_on='Region_code')

In [21]:
result_df_klein.dropna()

,Unnamed: 0,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased,Title,Totale bevolking,...,Overig niet-westerse migratieachtergrond.1,Sterfte,"Sterfte, relatief",Bevolkingsdichtheid,Woningdichtheid,Koopwoningen,Huurwoningen,Region_code,Vaccination_coverage_partly,Vaccination_coverage_completed
0,221848,2021-11-10 10:00:00,GM0014,Groningen,Groningen,22852,119,50,GM0014,232874.0,...,7.7,1805.0,7.7,1255.0,627.0,41.2,58.1,GM0014,84,82
1,221849,2021-11-10 10:00:00,GM0034,Almere,Flevoland,27178,392,113,GM0034,211893.0,...,12.8,1084.0,5.1,1640.0,666.0,63.9,35.9,GM0034,76,75
2,221850,2021-11-10 10:00:00,GM0037,Stadskanaal,Groningen,3052,31,21,GM0037,31686.0,...,2.4,440.0,13.9,269.0,130.0,59.5,39.3,GM0037,84,82
3,221851,2021-11-10 10:00:00,GM0047,Veendam,Groningen,2383,13,11,GM0047,27384.0,...,2.8,310.0,11.3,360.0,169.0,65.0,34.8,GM0047,83,81
4,221852,2021-11-10 10:00:00,GM0050,Zeewolde,Flevoland,2323,22,14,GM0050,22653.0,...,3.7,115.0,5.1,92.0,35.0,70.5,29.4,GM0050,82,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,222194,2021-11-10 10:00:00,GM1963,Hoeksche Waard,Zuid-Holland,11491,165,68,GM1963,87401.0,...,2.8,858.0,9.8,325.0,141.0,70.2,29.6,GM1963,83,81
347,222195,2021-11-10 10:00:00,GM1966,Het Hogeland,Groningen,4959,40,20,GM1966,47801.0,...,2.5,495.0,10.4,99.0,47.0,67.0,32.8,GM1966,84,83
348,222196,2021-11-10 10:00:00,GM1969,Westerkwartier,Groningen,4889,39,26,GM1969,63329.0,...,2.3,592.0,9.3,175.0,75.0,71.7,28.0,GM1969,86,85
349,222197,2021-11-10 10:00:00,GM1970,Noardeast-Fryslân,Fryslân,5297,69,42,GM1970,45228.0,...,2.1,485.0,10.7,120.0,55.0,66.4,33.0,GM1970,80,78


In [19]:
result_df_klein.dropna().to_csv('Data/Combi_df_klein.csv')